In [3]:
import os
import glob
import re
from datetime import datetime
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
from PIL import Image, ImageDraw, ImageFont
import imageio
import warnings
import csv # For writing the CSV file

# --- Helper function to parse date from filename ---
def get_date_from_filename(filename):
    """Extracts YYYY_MM from common GEE export filenames."""
    match = re.search(r'_(\d{4})_(\d{2})\.tif$', os.path.basename(filename))
    if match:
        year, month = map(int, match.groups())
        return datetime(year, month, 1)
    else:
        print(f"Warning: Could not parse date from filename: {filename}. Skipping.")
        return None

# --- Main Function ---
def create_nightlight_timelapse_and_graph(
    input_folder: str,
    output_path_base: str,
    output_format: str = 'gif',
    cmap_name: str = 'plasma',
    fps: int = 6,
    normalize_animation: bool = False, # <<< New parameter
    graph_title: str = 'Average Night Light Intensity Over Time',
    watermark_text: str = 'My Custom Watermark',
    watermark_size: int = 20,
    watermark_position: tuple = (10, 10)
    ):
    """
    Creates a timelapse animation, a time series graph, and a CSV data file
    from a folder of GeoTIFF night light images.

    Args:
        input_folder: Path to the directory containing the GeoTIFF files.
        output_path_base: Base path and filename for outputs (animation, graph, csv).
        output_format: 'gif' or 'mp4'. Defaults to 'gif'.
        cmap_name: Name of the matplotlib colormap for the animation.
        fps: Frames per second for the animation.
        normalize_animation: If True, normalize animation frames using the 0.5-99.5th
                             percentile range across all data. If False (default),
                             scale frames from 0 to the 99.5th percentile max value
                             for potentially better visualization of raw changes,
                             but less resilience to outliers affecting the max.
        graph_title: Title for the time series graph (uses raw data).
        watermark_text: Text for the watermark overlay on video frames.
        watermark_size: Approximate font size for the watermark (requires TTF for accuracy).
        watermark_position: Tuple (x, y) for the top-left corner of the watermark.
    """

    print(f"Starting analysis for folder: {input_folder}")
    print(f"Output base: {output_path_base}")
    print(f"Format: {output_format}, FPS: {fps}, Colormap: {cmap_name}")
    print(f"Normalize Animation Frames: {normalize_animation}") # <<< Log new option

    # --- 1. Find and Sort TIFF Files ---
    search_pattern = os.path.join(input_folder, '*.tif')
    tif_files = glob.glob(search_pattern)
    if not tif_files:
        print(f"Error: No .tif files found in {input_folder}")
        return

    file_date_pairs = [(get_date_from_filename(f), f) for f in tif_files]
    file_date_pairs = [p for p in file_date_pairs if p[0] is not None]
    if not file_date_pairs:
        print(f"Error: Could not parse dates from any filenames in {input_folder}")
        return

    file_date_pairs.sort()
    sorted_files = [f[1] for f in file_date_pairs]
    sorted_dates = [f[0] for f in file_date_pairs]
    date_labels = [d.strftime('%Y-%m') for d in sorted_dates]

    print(f"Found and sorted {len(sorted_files)} TIFF files.")


    # --- 2. Process Files: Calculate Raw Stats & Determine Range ---
    # monthly_avg_intensity will store the RAW average values for graph and CSV
    monthly_avg_intensity = []
    all_valid_data_list = []
    files_with_valid_data_count = 0
    max_raw_value_overall = -np.inf

    print("\n--- Pass 1: Calculating Raw Statistics & Preparing Range ---")
    for i, filepath in enumerate(sorted_files):
        # print(f"\n--- Processing file {i+1}/{len(sorted_files)}: {os.path.basename(filepath)} ---") # Verbose option
        try:
            with rasterio.open(filepath) as src:
                # Get Metadata & Raw Data
                nodata_val = src.nodata
                # try: dtype = src.dtype
                # except AttributeError: dtype = "Error reading dtype"
                # print(f"  Metadata: nodata={nodata_val}, dtype={dtype}, shape={src.shape}") # Verbose
                raw_data = src.read(1).astype(np.float32)

                # Update Max Raw Value
                with warnings.catch_warnings():
                     warnings.simplefilter("ignore", category=RuntimeWarning)
                     current_max_raw = np.nanmax(raw_data)
                if np.isfinite(current_max_raw) and current_max_raw > max_raw_value_overall:
                    max_raw_value_overall = current_max_raw

                # Manual Masking (for > 0)
                invalid_mask = (np.isnan(raw_data)) | (raw_data <= 0)
                # print(f"  Applied mask for non-positive values (<= 0) or existing NaNs.") # Verbose

                # Select VALID data
                valid_data = raw_data[~invalid_mask]
                valid_pixel_count = valid_data.size

                # print(f"  Manual Mask Result: valid_pixels_count={valid_pixel_count}") # Verbose
                if valid_pixel_count > 0:
                    files_with_valid_data_count += 1

                    # Calculate RAW Stats using NAN-ignoring functions
                    with warnings.catch_warnings():
                        warnings.simplefilter("ignore", category=RuntimeWarning)
                        min_val = np.nanmin(valid_data)
                        max_val = np.nanmax(valid_data)
                        mean_val = np.nanmean(valid_data)

                    if not (np.isfinite(min_val) and np.isfinite(max_val) and np.isfinite(mean_val)):
                         print(f"  WARNING: Stats calculation resulted in non-finite values for {os.path.basename(filepath)}. Appending NaN.")
                         monthly_avg_intensity.append(np.nan)
                    else:
                         # print(f"  Stats (VALID): min={min_val:.4f}, max={max_val:.4f}, mean={mean_val:.4f}") # Verbose
                         # Store the RAW mean for graph and CSV
                         monthly_avg_intensity.append(mean_val)
                         # Append data for calculating normalization range (needed regardless of normalize_animation setting)
                         all_valid_data_list.append(valid_data)

                else:
                    # print(f"  WARNING: No valid pixels found. Appending NaN.") # Verbose
                    monthly_avg_intensity.append(np.nan)

        except Exception as e:
            print(f"  ERROR reading or processing {os.path.basename(filepath)}: {e}")
            if len(monthly_avg_intensity) < i + 1:
                 monthly_avg_intensity.append(np.nan)

    print(f"\n--- Pass 1 Summary ---")
    print(f"  Processed {len(sorted_files)} files.")
    print(f"  Found valid data in {files_with_valid_data_count} files.")
    if not np.isfinite(max_raw_value_overall):
        max_raw_value_overall = 1.0 # Default if no finite max found
        print("  WARNING: Could not determine a finite overall maximum raw value. Using {max_raw_value_overall}.")
    else:
        print(f"  Overall maximum raw value encountered: {max_raw_value_overall:.4f}")


    # --- Determine Normalization Range (Calculated even if normalize_animation is False, as vmax is used) ---
    vis_vmin = 0.0  # Default minimum for visualization scaling
    vis_vmax = max(1.0, max_raw_value_overall) # Default maximum uses overall max as fallback

    if all_valid_data_list:
        print("  Determining visualization range using percentiles...")
        try:
            all_valid_data_list = [arr for arr in all_valid_data_list if isinstance(arr, np.ndarray) and arr.size > 0]
            if all_valid_data_list:
                 concatenated_data = np.concatenate(all_valid_data_list)
                 del all_valid_data_list # Free memory
                 print(f"  Total valid data points for range calculation: {concatenated_data.size}")

                 if concatenated_data.size > 0:
                     with warnings.catch_warnings():
                         warnings.simplefilter("ignore", category=RuntimeWarning)
                         p_low = np.nanpercentile(concatenated_data, 0.5)
                         p_high = np.nanpercentile(concatenated_data, 99.5)

                     # Use calculated percentiles if valid
                     if np.isfinite(p_low) and np.isfinite(p_high) and p_high > p_low:
                         vis_vmin = p_low
                         vis_vmax = p_high
                         print(f"  Using 0.5-99.5 percentile range for visualization: {vis_vmin:.4f} - {vis_vmax:.4f}")
                     else:
                         print("  Warning: Percentile calculation failed or gave invalid range. Using fallback range.")
                         # Fallback using min/max or overall max
                         min_fallback = np.nanmin(concatenated_data)
                         max_fallback = np.nanmax(concatenated_data)
                         if np.isfinite(min_fallback) and np.isfinite(max_fallback) and max_fallback > min_fallback:
                              vis_vmin = min_fallback
                              vis_vmax = max_fallback
                              print(f"  Using min-max fallback range: {vis_vmin:.4f} - {vis_vmax:.4f}")
                         else:
                              vis_vmin = 0.0
                              vis_vmax = max(1.0, max_raw_value_overall)
                              print(f"  Using overall max fallback range: {vis_vmin:.4f} - {vis_vmax:.4f}")
                 else:
                     print("  Warning: Concatenated data empty. Using default range [0, 1].")
                     vis_vmin = 0.0
                     vis_vmax = 1.0
                 if 'concatenated_data' in locals(): del concatenated_data # Free memory
            else:
                print("  Warning: No valid data arrays found. Using default range [0, 1].")
                vis_vmin = 0.0
                vis_vmax = 1.0

        except MemoryError:
             print("  ERROR: MemoryError during range calculation. Using fallback range.")
             vis_vmin = 0.0
             vis_vmax = max(1.0, max_raw_value_overall)
        except Exception as e:
             print(f"  ERROR during range calculation: {e}. Using fallback range.")
             vis_vmin = 0.0
             vis_vmax = max(1.0, max_raw_value_overall)
    else:
        print("  Warning: No valid data extracted. Using default range [0, 1].")
        vis_vmin = 0.0
        vis_vmax = 1.0

    # --- Final Range Sanity Check for Visualization ---
    if not (np.isfinite(vis_vmin) and np.isfinite(vis_vmax)):
         vis_vmin = 0.0
         vis_vmax = 1.0
    elif vis_vmax <= vis_vmin:
        vis_vmax = vis_vmin + 1.0

    # Ensure vmin for visualization is not negative
    if vis_vmin < 0:
        vis_vmin = 0.0

    # Define the normalization based on the chosen setting
    if normalize_animation:
        print(f"--- Using NORMALIZED animation range (percentile based): vmin={vis_vmin:.4f}, vmax={vis_vmax:.4f} ---")
        norm = mcolors.Normalize(vmin=vis_vmin, vmax=vis_vmax)
    else:
        # Use a fixed 0 to vmax scaling for non-normalized view
        fixed_vmax = vis_vmax # Use the calculated 99.5 percentile max
        fixed_vmin = 0.0
        print(f"--- Using FIXED animation scaling (0 to 99.5 percentile max): vmin={fixed_vmin:.4f}, vmax={fixed_vmax:.4f} ---")
        norm = mcolors.Normalize(vmin=fixed_vmin, vmax=fixed_vmax)


    # --- 3. Save Raw Intensity Data to CSV ---
    csv_filename = f"{output_path_base}_intensity_data.csv"
    print(f"\nSaving raw average intensity data to {csv_filename}...")
    try:
        with open(csv_filename, 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(['Date', 'Average_Radiance']) # Header row
            count_written = 0
            for date_obj, intensity in zip(sorted_dates, monthly_avg_intensity):
                # Only write rows with valid intensity data
                if np.isfinite(intensity):
                    date_str = date_obj.strftime('%Y-%m-%d') # Format date as YYYY-MM-DD
                    csvwriter.writerow([date_str, f"{intensity:.6f}"]) # Write date string and formatted intensity
                    count_written += 1
            print(f"  Successfully wrote {count_written} data points to CSV.")
    except Exception as e:
        print(f"  ERROR writing CSV file: {e}")


    # --- 4. Prepare Frames (Pass 2) ---
    frames = []
    cmap = cm.get_cmap(cmap_name)
    font = None
    try:
        font = ImageFont.load_default()
        print(f"Using default PIL font for watermark/date.")
    except IOError:
        print("Warning: Default PIL font not found. Watermark/Date text might not appear.")

    print("\n--- Pass 2: Creating animation frames ---")
    for i, filepath in enumerate(sorted_files):
        # print(f"  Processing frame {i+1}/{len(sorted_files)}: {os.path.basename(filepath)}") # Verbose
        try:
            with rasterio.open(filepath) as src:
                data = src.read(1).astype(np.float32)

                # Apply NaN mask consistent with Pass 1 (values <= 0 or existing NaN)
                nan_mask = (np.isnan(data)) | (data <= 0)
                data[nan_mask] = np.nan # Use NaN for invalid pixels

                # Normalize/Scale using the chosen 'norm' object (either percentile-based or fixed 0-vmax)
                # Map NaN values to the MINIMUM value of the normalization range for coloring
                data_processed = norm(np.nan_to_num(data, nan=norm.vmin))
                rgba_image = cmap(data_processed)

                # Optional Transparency
                # rgba_image[nan_mask, 3] = 0

                pil_image = Image.fromarray((rgba_image[:, :, :3] * 255).astype(np.uint8))

                # Draw Watermark/Date
                draw = ImageDraw.Draw(pil_image)
                if font:
                    if watermark_text:
                        try: draw.text(watermark_position, watermark_text, fill=(255, 255, 255, 180), font=font)
                        except Exception as text_err: print(f"    Warning: Error drawing watermark: {text_err}")
                    date_str = date_labels[i]
                    try:
                        bbox = draw.textbbox((0,0), date_str, font=font)
                        text_width = bbox[2] - bbox[0]; text_height = bbox[3] - bbox[1]
                        date_pos = (pil_image.width - text_width - 10, pil_image.height - text_height - 10)
                        draw.text(date_pos, date_str, fill=(255, 255, 255, 200), font=font)
                    except (AttributeError, TypeError): # Fallback
                        text_width = len(date_str) * 6; text_height = 10
                        date_pos = (pil_image.width - text_width - 10, pil_image.height - text_height - 10)
                        try: draw.text(date_pos, date_str, fill=(255, 255, 255, 200), font=font)
                        except Exception as text_err: print(f"    Warning: Error drawing date: {text_err}")

                frames.append(np.array(pil_image))
        except Exception as e:
            print(f"  ERROR creating frame for {os.path.basename(filepath)}: {e}")

    # --- 5. Create Animation ---
    if frames:
        output_filename_anim = f"{output_path_base}.{output_format.lower()}"
        print(f"\nSaving animation to {output_filename_anim}...")
        try:
            if output_format.lower() == 'gif':
                 imageio.mimsave(output_filename_anim, frames, format=output_format, duration=int(1000/fps)) # duration in ms
            elif output_format.lower() == 'mp4':
                 imageio.mimsave(output_filename_anim, frames, format=output_format, fps=fps, quality=8, macro_block_size=16) # Added macro_block_size
            else:
                 imageio.mimsave(output_filename_anim, frames, format=output_format, fps=fps)
            print("Animation saved successfully.")
        except Exception as e:
            print(f"Error saving animation: {e}")
    else:
        print("\nNo frames generated, skipping animation saving.")

    # --- 6. Create and Save Graph (using RAW data) ---
    output_filename_graph = f"{output_path_base}_graph.png"
    print(f"\nSaving graph of raw intensity data to {output_filename_graph}...")
    # Use finite values from the RAW monthly_avg_intensity list
    valid_indices = np.isfinite(monthly_avg_intensity)
    plot_dates = np.array(sorted_dates)[valid_indices]
    plot_intensity_raw = np.array(monthly_avg_intensity)[valid_indices]
    print(f"  Number of valid data points for graph: {len(plot_dates)}")

    if len(plot_dates) > 0:
        fig, ax = plt.subplots(figsize=(15, 7))
        ax.plot(plot_dates, plot_intensity_raw, marker='.', linestyle='-', markersize=5, label='Monthly Avg Radiance (Raw)')

        # Optional: Rolling average (requires pandas)
        # try:
        #     import pandas as pd
        #     ts = pd.Series(plot_intensity_raw, index=pd.to_datetime(plot_dates))
        #     rolling_avg = ts.rolling(window=12, center=True).mean()
        #     ax.plot(rolling_avg.index, rolling_avg.values, color='red', linestyle='--', linewidth=1.5, label='12-Month Rolling Avg')
        # except ImportError: pass

        ax.xaxis.set_major_locator(mdates.YearLocator(2))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        ax.xaxis.set_minor_locator(mdates.MonthLocator(interval=6))
        fig.autofmt_xdate(rotation=45)
        ax.set_xlabel("Time")
        ax.set_ylabel("Average Radiance (Raw, e.g., nW/cm²/sr)")
        ax.set_title(graph_title)
        ax.legend()
        ax.grid(True, which='major', linestyle='-', linewidth=0.6, alpha=0.7)
        ax.grid(True, which='minor', linestyle=':', linewidth=0.4, alpha=0.5)
        ax.set_ylim(bottom=0) # Start y-axis at 0
        plt.tight_layout()
        try:
            plt.savefig(output_filename_graph, dpi=300)
            print("Graph saved successfully.")
        except Exception as e:
            print(f"Error saving graph: {e}")
        plt.close(fig)
    else:
        print("No valid intensity data to plot, skipping graph saving.")

    print("\nProcessing finished.")





In [4]:
# --- Example Usage ---
if __name__ == "__main__":
    # --- Configuration ---
    GEE_DOWNLOAD_FOLDER = r'C:/Users/rodri/My Drive/GEE_NightLights_Venezuela' # <<< UPDATE
    OUTPUT_BASE_NAME = r'C:/Users/rodri/Desktop/Nightlights/Cuba_normalized/Venezuela_nightlight_analysis_with_csv' # <<< UPDATE

    # --- Path checks ---
    if not os.path.isdir(GEE_DOWNLOAD_FOLDER):
         print(f"ERROR: Input folder not found: {GEE_DOWNLOAD_FOLDER}")
         exit()
    output_dir = os.path.dirname(OUTPUT_BASE_NAME)
    if not os.path.isdir(output_dir):
         print(f"Output directory not found: {output_dir}")
         try:
             os.makedirs(output_dir, exist_ok=True)
             print(f"Created output directory: {output_dir}")
         except OSError as e:
             print(f"ERROR: Could not create output directory: {e}")
             exit()    # --- Run the function ---
    create_nightlight_timelapse_and_graph(
        input_folder=GEE_DOWNLOAD_FOLDER,
        output_path_base=OUTPUT_BASE_NAME,
        output_format='mp4',
        cmap_name='plasma',
        fps=10,
        normalize_animation=False, # <<< Set to False for fixed 0-vmax scaling, True for percentile normalization
        graph_title='Venezuela: Average Monthly Night Light Intensity (VIIRS DNB) -Normalized',
        watermark_text='Anaplian.com VIIRS Monthly - Venezuela',
        watermark_size= 40, # Limited effect with default font
        watermark_position=(15, 15)
    )


Starting analysis for folder: C:/Users/rodri/My Drive/GEE_NightLights_Venezuela
Output base: C:/Users/rodri/Desktop/Nightlights/Cuba_normalized/Venezuela_nightlight_analysis_with_csv
Format: mp4, FPS: 10, Colormap: plasma
Normalize Animation Frames: False
Found and sorted 155 TIFF files.

--- Pass 1: Calculating Raw Statistics & Preparing Range ---

--- Pass 1 Summary ---
  Processed 155 files.
  Found valid data in 155 files.
  Overall maximum raw value encountered: 233260.7188
  Determining visualization range using percentiles...
  Total valid data points for range calculation: 478122634
  Using 0.5-99.5 percentile range for visualization: 0.0038 - 21.7588
--- Using FIXED animation scaling (0 to 99.5 percentile max): vmin=0.0000, vmax=21.7588 ---

Saving raw average intensity data to C:/Users/rodri/Desktop/Nightlights/Cuba_normalized/Venezuela_nightlight_analysis_with_csv_intensity_data.csv...
  Successfully wrote 155 data points to CSV.
Using default PIL font for watermark/date.

-

C:\Users\rodri\AppData\Local\Temp\ipykernel_24656\3257055091.py:265: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap(cmap_name)



Saving animation to C:/Users/rodri/Desktop/Nightlights/Cuba_normalized/Venezuela_nightlight_analysis_with_csv.mp4...


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (3022, 2638) to (3024, 2640) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Animation saved successfully.

Saving graph of raw intensity data to C:/Users/rodri/Desktop/Nightlights/Cuba_normalized/Venezuela_nightlight_analysis_with_csv_graph.png...
  Number of valid data points for graph: 155
Graph saved successfully.

Processing finished.
